# Iteration 5: No. cancelled electives

## Plan English Summary

In this iteration we add in teh first basic performance measure calculation.

1. Number of cancelled elective operations.

These calculations only use statistics collected in the results collection period.


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_5.md"))

modify the code as follows:

The code should calculate performance measures at the end of the simulation run.  

The following measures should be calculated.

1. The number of elective operations cancelled. 

The code should only collect statistics on this performance measure if the simulation time has exceeded the warm-up period.

After the performance measures are calculated the results should outputted in a pandas dataframe.

Show all of the model code: this includes all patient generator functions in the CCModel class and all code in the Experiment class.





2. The total time critical care beds were in use represented as a percentage of the (number of beds multiplied by the results collection period). 
3. The mean time an unplanned admission had to wait for a critical care bed to be available.

## Date and time of input to LLM

> 1540 30th November 2023 

## Observations on the LLM Output

* The code appears to correctly add cancelled operations to the count if the warm-up period is completed.  I.e.

```python
if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
    if self.env.now > self.experiment.warm_up_period:
        self.cancelled_elective_operations += 1
```

* The code checks for warm-up period completion at the end of the run before printing out the statistics.  This was unexpected, but makes sense on one level as the no statistics will have been collection if the simulation has run less than the warm-up period.

## LLM Output 1:

In this modified code, the CCUModel class calculates the number of elective operations cancelled at the end of the simulation run. If the simulation time has exceeded the warm-up period, statistics on the performance measure are collected, and the results are outputted in a pandas dataframe.

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.cancelled_elective_operations = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.cancelled_elective_operations += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            df = pd.DataFrame({'Cancelled Elective Operations': [self.cancelled_elective_operations]})
            print(df)


In [4]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from Emergency surgery at time 6.808427251236074
Patient 1 admitted to critical care bed at time 6.808427251236074
Patient 2 arrived from Emergency surgery at time 15.152154844997206
Patient 2 admitted to critical care bed at time 15.152154844997206
Patient 3 arrived from the Wards at time 18.08207150614983
Patient 3 admitted to critical care bed at time 18.08207150614983
Elective surgery patient 4 arrived at time 19.042999221299905
Elective surgery patient 4 admitted to critical care bed at time 19.042999221299905
Patient 5 arrived from Accident and Emergency at time 19.70351908307115
Patient 5 admitted to critical care bed at time 19.70351908307115
Patient 5 discharged from critical care bed at time 23.29605223660634
Patient 5 discharged from critical care bed at time 26.87598723490114
Patient 6 arrived from Accident and Emergency at time 28.199870773532524
Patient 6 admitted to critical care bed at time 28.199870773532524
Critical care bed is available for next pat

## Modifications to LLM code to test for correct model outputs.

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [5]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [6]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [9]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [8]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 11.812819319606035
Patient 1 admitted to critical care bed at time 11.812819319606035
Patient 1 discharged from critical care bed at time 32.58873204350341
Critical care bed is available for next patient at time 37.58873204350341
Patient 2 arrived from Accident and Emergency at time 40.191220827521356
Patient 2 admitted to critical care bed at time 40.191220827521356
Patient 2 discharged from critical care bed at time 68.09305115490892
Critical care bed is available for next patient at time 73.09305115490892
Patient 3 arrived from Accident and Emergency at time 76.0777891666356
Patient 3 admitted to critical care bed at time 76.0777891666356
Patient 4 arrived from Accident and Emergency at time 110.84682943832419
Patient 4 admitted to critical care bed at time 110.84682943832419
Patient 4 discharged from critical care bed at time 143.08752104160465
Patient 5 arrived from Accident and Emergency at time 144.37517819391297
Patient 5 ad

### Test 2: Extreme value test 2

PASS

In [10]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [11]:
extreme_value_test_2()

Elective surgery patient 1 arrived at time 18.25996172418333
Elective surgery patient 1 admitted to critical care bed at time 18.25996172418333
Elective surgery patient 1 discharged from critical care bed at time 21.781707834281995
Critical care bed is available for next patient at time 26.781707834281995
Elective surgery patient 2 arrived at time 42.45177190184298
Elective surgery patient 2 admitted to critical care bed at time 42.45177190184298
Elective surgery patient 2 discharged from critical care bed at time 54.67986123926393
Elective surgery patient 3 arrived at time 58.652847009235785
Elective surgery patient 3 admitted to critical care bed at time 58.652847009235785
Critical care bed is available for next patient at time 59.67986123926393
Elective surgery patient 4 arrived at time 76.5251989366373
Elective surgery patient 4 admitted to critical care bed at time 76.5251989366373
Elective surgery patient 4 discharged from critical care bed at time 86.5426342806801
Critical care 

### Test 3: Different run length

PASS

In [12]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [13]:
test_results_collection_period(10)

Elective surgery patient 1 arrived at time 15.135282918625332
Elective surgery patient 1 admitted to critical care bed at time 15.135282918625332
Patient 2 arrived from Accident and Emergency at time 22.21110120935129
Patient 2 admitted to critical care bed at time 22.21110120935129
Patient 3 arrived from Emergency surgery at time 30.464636155003053
Patient 3 admitted to critical care bed at time 30.464636155003053
Patient 4 arrived from the Wards at time 30.58827883671311
Patient 4 admitted to critical care bed at time 30.58827883671311
Elective surgery patient 5 arrived at time 37.35328509415423
Elective surgery patient 5 admitted to critical care bed at time 37.35328509415423
Patient 6 arrived from Accident and Emergency at time 39.11063711688158
Patient 6 admitted to critical care bed at time 39.11063711688158
Patient 6 discharged from critical care bed at time 39.89599426707456
Critical care bed is available for next patient at time 44.89599426707456
Patient 7 arrived from Emergen

730

### Test 4: Extreme value test 3

PASS.

In [14]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [15]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 9.856755809612746
Patient 1 admitted to critical care bed at time 9.856755809612746
Patient 2 arrived from the Wards at time 12.283223838130835
Elective surgery patient 3 arrived at time 18.709585252360895
Elective surgery for patient 3 cancelled due to no available critical care beds at time 18.709585252360895
Patient 4 arrived from Accident and Emergency at time 21.450469308196833
Patient 5 arrived from Emergency surgery at time 31.1304975993091
Elective surgery patient 6 arrived at time 35.31228372714692
Elective surgery for patient 6 cancelled due to no available critical care beds at time 35.31228372714692
Patient 7 arrived from other hospitals at time 47.38749230623792
Elective surgery patient 8 arrived at time 52.29854564219239
Elective surgery for patient 8 cancelled due to no available critical care beds at time 52.29854564219239
Patient 9 arrived from Accident and Emergency at time 55.31182599791573
Patient 10 arrived from the 

### Test 5: Extreme value test 4

PASS

In [16]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [17]:
extreme_value_test_4()

Patient 1 arrived from other hospitals at time 3.1355876960219646
Patient 1 admitted to critical care bed at time 3.1355876960219646
Patient 2 arrived from the X-Ray department at time 7.921818866768826
Patient 2 admitted to critical care bed at time 7.921818866768826
Patient 3 arrived from Accident and Emergency at time 9.877171384398395
Patient 3 admitted to critical care bed at time 9.877171384398395
Patient 4 arrived from other hospitals at time 11.85665555477113
Patient 4 admitted to critical care bed at time 11.85665555477113
Patient 5 arrived from the Wards at time 13.707262099880417
Patient 5 admitted to critical care bed at time 13.707262099880417
Patient 6 arrived from Accident and Emergency at time 20.564121311449142
Patient 6 admitted to critical care bed at time 20.564121311449142
Elective surgery patient 7 arrived at time 21.63083771431923
Elective surgery patient 7 admitted to critical care bed at time 21.63083771431923
Patient 8 arrived from Emergency surgery at time 37

### Test 6: Vary Warm-up parameter

PASS

In [18]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [19]:
test_warmup_period(0)

Patient 1 arrived from Accident and Emergency at time 0.7421222376960996
Patient 1 admitted to critical care bed at time 0.7421222376960996
Patient 2 arrived from Accident and Emergency at time 1.3676242285708218
Patient 2 admitted to critical care bed at time 1.3676242285708218
Patient 3 arrived from other hospitals at time 7.14730971871494
Patient 3 admitted to critical care bed at time 7.14730971871494
Patient 3 discharged from critical care bed at time 10.308939177784456
Critical care bed is available for next patient at time 15.308939177784456
Elective surgery patient 4 arrived at time 19.374815105958195
Elective surgery patient 4 admitted to critical care bed at time 19.374815105958195
Patient 5 arrived from other hospitals at time 20.070763621213338
Patient 5 admitted to critical care bed at time 20.070763621213338
Patient 6 arrived from Emergency surgery at time 29.130989715341467
Patient 6 admitted to critical care bed at time 29.130989715341467
Patient 6 discharged from criti

(8640, 8640)